In [6]:
%config Completer.use_jedi = False

import sys
sys.path.insert(1,'../')


# Ignore all warnings
import warnings
warnings.filterwarnings('ignore')


# Pytorch related
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms
from torch.utils import data as dt
from torchinfo import summary
import torchvision.models as pretrained_models
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils import model_zoo
from torch.autograd import Variable



# Numpy, Matplotlib, Pandas, Sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score

from scipy.spatial import distance
from scipy.stats.stats import pearsonr
from scipy import stats

%matplotlib inline 


# python utilities
from itertools import combinations
import gc
import pickle
from tqdm import tqdm_notebook as tqdm
import copy
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import networkx as nx
import os
from IPython.display import Image
from IPython.core.debugger import set_trace
import collections
from functools import partial
import math
import time
import glob

from PIL import Image, ImageStat
from matplotlib.pyplot import imshow

# Extra imports
from lib.feature_extractor import FeatureExtractor
from lib.custom_dataset import Contour_Dataset
from lib.build_fe_ft_models import *
from lib.misc_functions import *
from lib.field_stim_functions import *




# Hyperparameters

In [7]:
img_dim=512

batch_size=32
num_workers=8
device = torch.device('cuda:'+'3')
# device = 'cpu'

print(device)

cuda:3


### Extract all the values from the config variables

In [8]:
from visualdiet_savedmodel_config import *

print('\n Visual Diet config \n')
print(visual_diet_config)


print('\n Saved Model config \n')
print(saved_model_config)




 Visual Diet config 

{'root_directory': '/home/jovyan/work/Datasets/contour_integration/model-training/config_0/', 'get_B': [0, 15, 30, 45, 60, 75], 'get_D': [32], 'get_A': [0], 'get_numElements': [12]}

 Saved Model config 

{'saved_model_directory_or_frozen_broad': '../../model_weights/contour_model_weights/alexnet_regimagenet_categ_frozen_broad/', 'saved_model_directory_or_finetune_broad': '../../model_weights/contour_model_weights/alexnet_regimagenet_categ_finetune_broad/', 'saved_model_directory_random_frozen_broad': '../../model_weights/contour_model_weights/alexnet-random-nodata-notask_frozen_broad/'}


In [9]:
root_directory=visual_diet_config['root_directory']
get_B=visual_diet_config['get_B']
get_D=visual_diet_config['get_D']
get_A=visual_diet_config['get_A']
get_numElements=visual_diet_config['get_numElements']

In [11]:
# model_file_paths = glob.glob(os.path.join(saved_directory,'*.pt'))

### Data Transforms

In [12]:
# normalize images using parameters from the training image set
data_transform = transforms.Compose([       
 transforms.Resize(img_dim),                   
 transforms.CenterCrop((img_dim,img_dim)),         
 transforms.ToTensor(),                    
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])

data_transform_without_norm = transforms.Compose([       
 transforms.Resize(img_dim),                   
 transforms.CenterCrop((img_dim,img_dim)),         
 transforms.ToTensor()                    
 ])

# Helper functions

In [13]:
def compute_diff_means(arr1, arr2):
    mean1 = np.mean(arr1)
    mean2 = np.mean(arr2)
    
    return (mean1 - mean2)

In [14]:
def compute_dprime_means(arr1, arr2):
    """
    Compute d' based on the difference of means divided by the pooled standard deviation.
    """
    mean1 = np.mean(arr1)
    mean2 = np.mean(arr2)
    
    var1 = np.var(arr1, ddof=1)  # ddof=1 for sample variance
    var2 = np.var(arr2, ddof=1)  # ddof=1 for sample variance
    
    # Calculate the pooled standard deviation
    pooled_std = np.sqrt((var1 + var2) / 2)
    
    # Compute d'
    d_prime = (mean1 - mean2) / pooled_std
    
    return d_prime

In [15]:
def compute_aprime_means(arr1, arr2):
    """
    Compute A' as the area under the ROC curve based on two input arrays.
    """
    # Combine the arrays and create corresponding labels
    combined = np.concatenate((arr1, arr2))
    labels = np.concatenate((np.ones(len(arr1)), np.zeros(len(arr2))))
    
    # Compute AUC
    a_prime = roc_auc_score(labels, combined)
    
    return a_prime

# Images

In [16]:
@interact(beta_val=(0,91,1),img_num=(0,20,1))
def get_data(beta_val=15,img_num=4):
    
    
    ### Show the image
    val_dataset_norm = Contour_Dataset(root=root_directory,transform=data_transform,train=False,get_B=[beta_val],get_D=get_D,get_A=get_A,get_numElements=get_numElements)
    val_loader_norm = torch.utils.data.DataLoader(dataset=val_dataset_norm, batch_size=batch_size, num_workers=num_workers, shuffle=False)

    val_dataset_without_norm = Contour_Dataset(root=root_directory,transform=data_transform_without_norm,train=False,get_B=[beta_val],get_D=get_D,get_A=get_A,get_numElements=get_numElements)
    val_loader_without_norm = torch.utils.data.DataLoader(dataset=val_dataset_without_norm, batch_size=batch_size, num_workers=num_workers, shuffle=False)


    
    
    a, b, d, alpha, nel, labels, record =next(iter(val_loader_norm))
    prep_img=torch.unsqueeze(a[img_num],0)
    print('Beta Val: ',b[img_num])
    print('Contour Present: ',labels[img_num])
    prep_img=prep_img.to(device)
    prep_img=Variable(prep_img,requires_grad=True)
    ## Plot the orginal image
    original_img=np.transpose(prep_img[0].detach().cpu().numpy(),(1,2,0))
    plt.figure(figsize=(10,8))
    plt.imshow(original_img)
    
    ## Plot the orginal unnormalized image
    plt.figure(figsize=(10,8))
    a_without_norm,b, d, alpha, nel, labels, record=next(iter(val_loader_without_norm))
    plt.imshow(np.transpose(torch.unsqueeze(a_without_norm[img_num],0)[0].numpy(),(1,2,0)))
    
    
    ## Plot the image with contour
    print(record[img_num])
    image_recorder_dict=torch.load(record[img_num])
    img_contour, img_control, img_contour_background, img_control_background=image_renderer(image_recorder_dict)
    display(blend(show_path(image_recorder_dict['path_points'], image_recorder_dict['path_centers'], imHeight=image_recorder_dict['image_height'], imWidth=image_recorder_dict['image_width']),img_contour_background,0.7))
    
    display(img_contour_background)
    display(img_contour)

    
    

interactive(children=(IntSlider(value=15, description='beta_val', max=91), IntSlider(value=4, description='img…

# 1) Analysis (Saliency) over 1 image

In [17]:
# @interact(beta_val=(0,91,1),img_num=(0,batch_size,1))
# def get_data(beta_val=15,img_num=4):
    
    
#     ### Show the image
#     val_dataset_norm = Contour_Dataset(root=root_directory,transform=data_transform,train=False,get_B=[beta_val],get_D=get_D,get_A=get_A,get_numElements=get_numElements)
#     val_loader_norm = torch.utils.data.DataLoader(dataset=val_dataset_norm, batch_size=batch_size, num_workers=num_workers, shuffle=False)

#     val_dataset_without_norm = Contour_Dataset(root=root_directory,transform=data_transform_without_norm,train=False,get_B=[beta_val],get_D=get_D,get_A=get_A,get_numElements=get_numElements)
#     val_loader_without_norm = torch.utils.data.DataLoader(dataset=val_dataset_without_norm, batch_size=batch_size, num_workers=num_workers, shuffle=False)


    
    
#     a, b, d, alpha, nel, labels, record =next(iter(val_loader_norm))
#     prep_img=torch.unsqueeze(a[img_num],0)
#     print('Beta Val: ',b[img_num])
#     print('Contour Present: ',labels[img_num])
#     prep_img=prep_img.to(device)
#     prep_img=Variable(prep_img,requires_grad=True)
    
#     ## Plot the orginal image - this is the normalized one, the one that will actually be used
#     # original_img=np.transpose(prep_img[0].detach().cpu().numpy(),(1,2,0))
#     # plt.figure(figsize=(10,8))
#     # plt.imshow(original_img)
#     # plt.close()
    
    
    
#     ## Plot the orginal unnormalized image
#     plt.figure(figsize=(10,8))
#     a_without_norm,b, d, alpha, nel, labels, record=next(iter(val_loader_without_norm))
#     plt.imshow(np.transpose(torch.unsqueeze(a_without_norm[img_num],0)[0].numpy(),(1,2,0)))
#     plt.axis('off')
#     # plt.savefig('./dev/ccn_figures/poster/original_image.png', bbox_inches='tight',dpi=300) 
#     # plt.close()
    
    
    
    
#     @interact(file=model_file_paths)
#     def vis_image(file,show_vis=False):
        

#         if(show_vis):    
#             checkpoint=torch.load(file)
            
#             ####################################################################################################
#             ## Showing the Model Stats
#             print('BASE MODEL IS',checkpoint['training_config']['base_model_name'])
#             print('Final Val Accuracy is',checkpoint['metrics']['val_acc'][-1])

#             plt.figure(figsize=(12,4))


#             plt.subplot(1,2,1)
#             plt.plot(np.arange(len(checkpoint['metrics']['train_acc'])),checkpoint['metrics']['train_acc'],label='train')
#             plt.plot(np.arange(len(checkpoint['metrics']['val_acc'])),checkpoint['metrics']['val_acc'],label='val')
#             plt.axhline(y=0.5,linestyle='--',color='r',label='chance')

#             plt.xlabel('Epochs',fontsize=15,labelpad=15)
#             plt.ylabel('Accuracy',fontsize=15,labelpad=15)

#             plt.gca().spines['top'].set_visible(False)
#             plt.gca().spines['right'].set_visible(False)
#             plt.yticks(fontsize=15)
#             plt.xticks(fontsize=15)   
#             plt.ylim(0.4,1.01)
#             plt.subplot(1,2,2)
#             plt.plot(np.arange(len(checkpoint['metrics']['train_loss'])),checkpoint['metrics']['train_loss'],label='train')
#             plt.plot(np.arange(len(checkpoint['metrics']['val_loss'])),checkpoint['metrics']['val_loss'],label='val')

#             plt.xlabel('Epochs',fontsize=15,labelpad=15)
#             plt.ylabel('Loss',fontsize=15,labelpad=15)

#             plt.gca().spines['top'].set_visible(False)
#             plt.gca().spines['right'].set_visible(False)
#             plt.yticks(fontsize=15)
#             plt.xticks(fontsize=15)   
#             plt.legend()
#             plt.show()
#             ####################################################################################################
            
            


#             ####################################################################################################
#             ## Loading the model and setting up the Guided Spliced Model for Attention Maps
#             loaded_spliced_model=SpliceModel(checkpoint['training_config']['base_model_name'],checkpoint['training_config']['layer_name'],fine_tune=checkpoint['training_config']['fine_tune'],device=device)
#             loaded_spliced_model.load_state_dict(checkpoint['model_state_dict'],strict=False)
#             GBP = Spliced_GuidedBackprop(loaded_spliced_model,device)
#             ####################################################################################################
            
            
            
            
#             ####################################################################################################
#             ## Getting the Saliency Maps
#             # Get gradients
#             guided_grads = GBP.generate_gradients(prep_img, 1)

            
#             # Gradients
#             guided_grads_vis = guided_grads - guided_grads.min()
#             guided_grads_vis /= guided_grads_vis.max()

#             plt.figure(figsize=(10,8))
#             plt.imshow(np.transpose(guided_grads_vis,(1,2,0)))
#             plt.axis('off')
#             plt.show()


#             # Grayscale Gradients
#             grayscale_guided_grads = convert_to_grayscale(guided_grads)


#             grayscale_guided_grads_vis = grayscale_guided_grads - grayscale_guided_grads.min()
#             grayscale_guided_grads_vis /= grayscale_guided_grads_vis.max()

#             plt.figure(figsize=(10,8))
#             plt.imshow(np.squeeze(np.transpose(grayscale_guided_grads_vis,(1,2,0))),cmap='gray')
#             plt.axis('off')
#             plt.show()
            
#             # plt.savefig('./dev/ccn_figures/poster/saliency_image.png',bbox_inches='tight', dpi=300) 
#             # plt.close()




#             ## Compute saliency maps
#             print(grayscale_guided_grads_vis.shape)
#             viz_image=grayscale_guided_grads_vis[0]
#             ####################################################################################################
            
            
            
            
            
#             ####################################################################################################
#             ## Compute saliency scores
#             mask_img_path_fg, mask_img_path_bg, list_gauss=mask_renderer(torch.load(record[img_num]),140)
#             mask_img_path_fg=np.array(mask_img_path_fg)
#             mask_img_path_bg=np.array(mask_img_path_bg)
            
#             plt.figure(figsize=(20,8))
#             plt.subplot(1,2,1)
#             plt.imshow(mask_img_path_fg,cmap='gray')
            
#             plt.subplot(1,2,2)
#             plt.imshow(mask_img_path_bg,cmap='gray')
#             plt.axis('off')
#             plt.show()
            
            
#             ## Plotting Gradients for contour and background elements
#             sns.distplot(viz_image[np.where(mask_img_path_fg==255)],label='distribution of fg')
#             sns.distplot(viz_image[np.where(mask_img_path_bg==255)],label='distribution of bg')
#             plt.title('Distribution of gradients')
#             plt.legend()
#             plt.gca().spines['top'].set_visible(False)
#             plt.gca().spines['right'].set_visible(False)
#             plt.show()
    

            
            
#             saliency_score_diffmean=compute_diff_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
#             saliency_score_dprime=compute_dprime_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
#             saliency_score_aprime=compute_aprime_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
            
            
#             print('saliency_score_diffmean: \t', saliency_score_diffmean)
#             print('saliency_score_diffmean: \t', saliency_score_dprime)
#             print('saliency_score_diffmean: \t',saliency_score_aprime)
#             ####################################################################################################
            


# 2) Analysis (Saliency) over all images from the validation set (also you will have to pre select the model). 

This analysis takes roughly 25 minutes to complete on 600 validation images when run on the contour model reading out from avgpool layer of Alexnet

## Select the dataset

In [18]:
# val_dataset_norm = Contour_Dataset(root=root_directory,transform=data_transform,train=False,get_B=get_B,get_D=get_D,get_A=get_A,get_numElements=get_numElements)
# # val_loader_norm = torch.utils.data.DataLoader(dataset=val_dataset_norm, batch_size=batch_size, num_workers=num_workers, shuffle=False)
# val_loader_norm = torch.utils.data.DataLoader(dataset=val_dataset_norm, batch_size=1, num_workers=num_workers, shuffle=False)
# print(len(val_dataset_norm))

## Select the model from the saved model directory

In [19]:
# for i,file in enumerate(model_file_paths):
#     print(i,file)

In [20]:
# selected_file=model_file_paths[0]
# print(selected_file)

## Computing the Saliency for all images

In [21]:
# def compute_saliency(prep_img,prep_recorder,file):
#     prep_img=Variable(prep_img,requires_grad=True).to(device)

    
    
    
#     ####################################################################################################
#     ## Loading the model and setting up the Guided Spliced Model for Attention Maps   
#     ## Model
#     checkpoint=torch.load(selected_file)
    
#     loaded_spliced_model=SpliceModel(checkpoint['training_config']['base_model_name'],checkpoint['training_config']['layer_name'],fine_tune=checkpoint['training_config']['fine_tune'],device=device)

#     loaded_spliced_model.load_state_dict(checkpoint['model_state_dict'],strict=False)
#     GBP = Spliced_GuidedBackprop(loaded_spliced_model,device)
#     ####################################################################################################

    
    
    
    

#     ####################################################################################################
#     ## Getting the Saliency Maps
#     # Get gradients
#     guided_grads = GBP.generate_gradients(prep_img, 1)
#     # Gradients
#     guided_grads_vis = guided_grads - guided_grads.min()
#     guided_grads_vis /= guided_grads_vis.max()

#     # Grayscale Gradients
#     grayscale_guided_grads = convert_to_grayscale(guided_grads)
#     grayscale_guided_grads_vis = grayscale_guided_grads - grayscale_guided_grads.min()
#     grayscale_guided_grads_vis /= grayscale_guided_grads_vis.max()
    
    
#     viz_image=grayscale_guided_grads_vis[0]
#     ####################################################################################################

    
    
    
    
#     ####################################################################################################
#     ## Compute saliency scores
#     mask_img_path_fg, mask_img_path_bg, list_gauss=mask_renderer(torch.load(prep_recorder),140)
#     mask_img_path_fg=np.array(mask_img_path_fg)
#     mask_img_path_bg=np.array(mask_img_path_bg)
    
    
    
#     saliency_score_diffmean=compute_diff_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
#     saliency_score_dprime=compute_dprime_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
#     saliency_score_aprime=compute_aprime_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
#     ####################################################################################################
    
    
    
    
    
#     ####################################################################################################
#     ## Making the run more memory efficient
#     loaded_spliced_model=loaded_spliced_model.to('cpu')
#     del GBP
#     torch.cuda.empty_cache()
#     gc.collect()
#     ####################################################################################################
    
    
    
#     return saliency_score_diffmean,saliency_score_dprime,saliency_score_aprime

In [22]:
# all_saliency_score_diff=[]
# all_saliency_score_dprime=[]
# all_saliency_score_aprime=[]

# all_betas=[]
# all_labels=[]

# for (a, b, d, alpha, nel, labels, record) in tqdm(val_loader_norm):
#     for img_num in tqdm(range(a.shape[0]),disable=True):
        
#         prep_img=torch.unsqueeze(a[img_num],0)
#         prep_recorder=record[img_num]
        
#         saliency_score_diffmean,saliency_score_dprime,saliency_score_aprime=compute_saliency(prep_img,prep_recorder,file)
        
#         all_saliency_score_diff.append(saliency_score_diffmean)
#         all_saliency_score_dprime.append(saliency_score_dprime)
#         all_saliency_score_aprime.append(saliency_score_aprime)
        
        
        
#     all_betas.append(b)
#     all_labels.append(labels)

# all_betas=torch.cat(all_betas).numpy()
# all_labels=torch.cat(all_labels).numpy()
# all_saliency_score_diff=np.array(all_saliency_score_diff)
# all_saliency_score_dprime=np.array(all_saliency_score_dprime)
# all_saliency_score_aprime=np.array(all_saliency_score_aprime)

## Plotting the scores

In [23]:
# contour_present_pos=np.where(all_labels==1)[0]
# contour_absent_pos=np.where(all_labels==0)[0]


## Alignment Sensitivity via Diff. of Means Measure

In [24]:
# # Create a figure
# fig = plt.figure(figsize=(16,10))
# # Create a GridSpec object
# gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1], height_ratios=[1, 1])



# # First subplot: Left column, spanning both rows
# ax1 = plt.subplot(gs[:, 0])

# # Second subplot: Top-right corner
# ax2 = plt.subplot(gs[0, 1])

# # Third subplot: Bottom-right corner
# ax3 = plt.subplot(gs[1, 1])






# # First subplot: Left column, spanning both rows
# # Scatter Plot with Connecting Lines
# for x1, x2 in zip(all_saliency_score_diff[contour_absent_pos], all_saliency_score_diff[contour_present_pos]):
#     ax1.plot([1, 4], [x1, x2], 'ko-', markersize=8,alpha=0.09)
# ax1.set_xticks([1, 4], ['Contour absent', 'Contour present'])
# ax1.set_xlim(0,5)
# ax1.set_ylabel('saliency_score_diffmean')
# ax1.set_title('Contour Sensitivity using Diff. Means')
# ax1.spines['top'].set_visible(False)
# ax1.spines['right'].set_visible(False)






# # Second subplot: Top-right corner
# sns.distplot(all_saliency_score_diff[contour_present_pos], ax=ax2, label='contour_present',color='green')
# sns.distplot(all_saliency_score_diff[contour_absent_pos], ax=ax2, label='contour_absent',color='gray')
# ax2.set_title('Contour Sensitivity using Diff. Means (Histogram)')
# ax2.legend()
# ax2.spines['top'].set_visible(False)
# ax2.spines['right'].set_visible(False)





# # Third subplot: Bottom-right corner
# sns.distplot(all_saliency_score_diff[contour_present_pos] - all_saliency_score_diff[contour_absent_pos], ax=ax3, label='difference',color='k')
# ax3.set_title('Alignment Sensitivity using Diff. Means')
# ax3.spines['top'].set_visible(False)
# ax3.spines['right'].set_visible(False)
# plt.show()



# # Perform the paired t-test
# t_statistic, p_value = stats.ttest_rel(all_saliency_score_diff[contour_present_pos], all_saliency_score_diff[contour_absent_pos])
# print(f"t-statistic: {t_statistic:.4f}")
# print(f"p-value: {p_value:.4f}")

## Alignment Sensitivity via DPrime Measure

In [25]:
# # Create a figure
# fig = plt.figure(figsize=(16,10))
# # Create a GridSpec object
# gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1], height_ratios=[1, 1])



# # First subplot: Left column, spanning both rows
# ax1 = plt.subplot(gs[:, 0])

# # Second subplot: Top-right corner
# ax2 = plt.subplot(gs[0, 1])

# # Third subplot: Bottom-right corner
# ax3 = plt.subplot(gs[1, 1])






# # First subplot: Left column, spanning both rows
# # Scatter Plot with Connecting Lines
# for x1, x2 in zip(all_saliency_score_dprime[contour_absent_pos], all_saliency_score_dprime[contour_present_pos]):
#     ax1.plot([1, 4], [x1, x2], 'ko-', markersize=8,alpha=0.09)
# ax1.set_xticks([1, 4], ['Contour absent', 'Contour present'])
# ax1.set_xlim(0,5)
# ax1.set_ylabel('saliency_score_dprime')
# ax1.set_title('Contour Sensitivity using DPrime')
# ax1.spines['top'].set_visible(False)
# ax1.spines['right'].set_visible(False)






# # Second subplot: Top-right corner
# sns.distplot(all_saliency_score_dprime[contour_present_pos], ax=ax2, label='contour_present',color='green')
# sns.distplot(all_saliency_score_dprime[contour_absent_pos], ax=ax2, label='contour_absent',color='gray')
# ax2.set_title('Contour Sensitivity using DPrime (Histogram)')
# ax2.legend()
# ax2.spines['top'].set_visible(False)
# ax2.spines['right'].set_visible(False)





# # Third subplot: Bottom-right corner
# sns.distplot(all_saliency_score_dprime[contour_present_pos] - all_saliency_score_dprime[contour_absent_pos], ax=ax3, label='difference',color='k')
# ax3.set_title('Alignment Sensitivity using DPrime')
# ax3.spines['top'].set_visible(False)
# ax3.spines['right'].set_visible(False)
# plt.show()



# # Perform the paired t-test
# t_statistic, p_value = stats.ttest_rel(all_saliency_score_dprime[contour_present_pos], all_saliency_score_dprime[contour_absent_pos])
# print(f"t-statistic: {t_statistic:.4f}")
# print(f"p-value: {p_value:.4f}")

## Alignment Sensitivity via APrime Measure

In [26]:
# # Create a figure
# fig = plt.figure(figsize=(16,10))
# # Create a GridSpec object
# gs = gridspec.GridSpec(2, 2, width_ratios=[3, 1], height_ratios=[1, 1])



# # First subplot: Left column, spanning both rows
# ax1 = plt.subplot(gs[:, 0])

# # Second subplot: Top-right corner
# ax2 = plt.subplot(gs[0, 1])

# # Third subplot: Bottom-right corner
# ax3 = plt.subplot(gs[1, 1])






# # First subplot: Left column, spanning both rows
# # Scatter Plot with Connecting Lines
# for x1, x2 in zip(all_saliency_score_aprime[contour_absent_pos], all_saliency_score_aprime[contour_present_pos]):
#     ax1.plot([1, 4], [x1, x2], 'ko-', markersize=8,alpha=0.09)
# ax1.set_xticks([1, 4], ['Contour absent', 'Contour present'])
# ax1.set_xlim(0,5)
# ax1.set_ylabel('saliency_score_aprime')
# ax1.set_title('Contour Sensitivity using APrime')
# ax1.spines['top'].set_visible(False)
# ax1.spines['right'].set_visible(False)






# # Second subplot: Top-right corner
# sns.distplot(all_saliency_score_aprime[contour_present_pos], ax=ax2, label='contour_present',color='green')
# sns.distplot(all_saliency_score_aprime[contour_absent_pos], ax=ax2, label='contour_absent',color='gray')
# ax2.set_title('Contour Sensitivity using APrime (Histogram)')
# ax2.legend()
# ax2.spines['top'].set_visible(False)
# ax2.spines['right'].set_visible(False)





# # Third subplot: Bottom-right corner
# sns.distplot(all_saliency_score_aprime[contour_present_pos] - all_saliency_score_aprime[contour_absent_pos], ax=ax3, label='difference',color='k')
# ax3.set_title('Alignment Sensitivity using APrime')
# ax3.spines['top'].set_visible(False)
# ax3.spines['right'].set_visible(False)
# plt.show()



# # Perform the paired t-test
# t_statistic, p_value = stats.ttest_rel(all_saliency_score_aprime[contour_present_pos], all_saliency_score_aprime[contour_absent_pos])
# print(f"t-statistic: {t_statistic:.4f}")
# print(f"p-value: {p_value:.4f}")

### Beta-wise saliency score

In [27]:
# unique_beta_vals=np.unique(all_betas)
# unique_beta_saliency_mean=[]
# unique_beta_saliency_std=[]



# for i in unique_beta_vals:
    
#     unique_beta_saliency_mean.append(np.mean(all_saliency_score_aprime[np.where(all_betas==i)[0]]))
#     unique_beta_saliency_std.append(np.std(all_saliency_score_aprime[np.where(all_betas==i)[0]],ddof=1))

In [28]:
# print(unique_beta_saliency_mean)
# print(unique_beta_saliency_std)

In [29]:
# ## ALREADY COMPUTED OVERALL
# height=unique_beta_saliency_mean
# error_bar=unique_beta_saliency_std

# bars=unique_beta_vals
# y_pos = np.arange(len(bars)) / 3.5
# color=['green'] * len(unique_beta_vals)

# plt.figure(figsize=(12,4))

# plt.gca().bar(y_pos, height, yerr=error_bar, align='center', width=0.09, color=color, alpha=0.2, ecolor='gray', capsize=4)
# plt.xticks(y_pos, bars,fontsize=10)
# plt.yticks(fontsize=10)
# plt.ylim(0.0,1.0)
# plt.ylabel('Saliency Score',fontsize=15,labelpad=20)
# plt.xlabel('Beta Values',fontsize=15,labelpad=20)
# plt.gca().spines['top'].set_visible(False)
# plt.gca().spines['right'].set_visible(False)
# plt.gca().spines['left'].set_visible(True)
# plt.gca().spines['bottom'].set_visible(True)

# 3) NEW-All models, All images

# Location sensitivity and Alignment sensitivity

This analysis takes roughly 8 hours minutes to complete on 600 validation images when run on all contour readout models (with object recognition alexnet finetuned backbones and trained on braod contour curvatures) 

## Setup the dataset and set of models

In [47]:
val_dataset_norm = Contour_Dataset(root=root_directory,transform=data_transform,train=False,get_B=get_B,get_D=get_D,get_A=get_A,get_numElements=get_numElements)
val_loader_norm = torch.utils.data.DataLoader(dataset=val_dataset_norm, batch_size=batch_size, num_workers=num_workers, shuffle=False)
print(len(val_dataset_norm))

600


In [48]:
model_file_paths=sorted(glob.glob(os.path.join(saved_model_config['saved_model_directory_or_finetune_broad'],'*.pt')))

In [49]:
for i,file in enumerate(model_file_paths):
    print(i,file)

0 ../../model_weights/contour_model_weights/alexnet_regimagenet_categ_finetune_broad/model_alexnet-pytorch-regim-categ_layer_avgpool_mode_finetune.pt
1 ../../model_weights/contour_model_weights/alexnet_regimagenet_categ_finetune_broad/model_alexnet-pytorch-regim-categ_layer_classifier-0_mode_finetune.pt
2 ../../model_weights/contour_model_weights/alexnet_regimagenet_categ_finetune_broad/model_alexnet-pytorch-regim-categ_layer_classifier-1_mode_finetune.pt
3 ../../model_weights/contour_model_weights/alexnet_regimagenet_categ_finetune_broad/model_alexnet-pytorch-regim-categ_layer_classifier-2_mode_finetune.pt
4 ../../model_weights/contour_model_weights/alexnet_regimagenet_categ_finetune_broad/model_alexnet-pytorch-regim-categ_layer_classifier-3_mode_finetune.pt
5 ../../model_weights/contour_model_weights/alexnet_regimagenet_categ_finetune_broad/model_alexnet-pytorch-regim-categ_layer_classifier-4_mode_finetune.pt
6 ../../model_weights/contour_model_weights/alexnet_regimagenet_categ_finet

## Computing the Saliency for all images

In [51]:
def compute_saliency(prep_img,prep_recorder,file):
    prep_img=Variable(prep_img,requires_grad=True).to(device)

    
    
    
    ####################################################################################################
    ## Loading the model and setting up the Guided Spliced Model for Attention Maps   
    ## Model
    checkpoint=torch.load(selected_file)
    
    loaded_spliced_model=SpliceModel(checkpoint['training_config']['base_model_name'],checkpoint['training_config']['layer_name'],fine_tune=checkpoint['training_config']['fine_tune'],device=device)

    loaded_spliced_model.load_state_dict(checkpoint['model_state_dict'],strict=False)
    GBP = Spliced_GuidedBackprop(loaded_spliced_model,device)
    ####################################################################################################

    
    
    
    

    ####################################################################################################
    ## Getting the Saliency Maps
    # Get gradients
    guided_grads = GBP.generate_gradients(prep_img, 1)
    # Gradients
    guided_grads_vis = guided_grads - guided_grads.min()
    guided_grads_vis /= guided_grads_vis.max()

    # Grayscale Gradients
    grayscale_guided_grads = convert_to_grayscale(guided_grads)
    grayscale_guided_grads_vis = grayscale_guided_grads - grayscale_guided_grads.min()
    grayscale_guided_grads_vis /= grayscale_guided_grads_vis.max()
    
    
    viz_image=grayscale_guided_grads_vis[0]
    ####################################################################################################

    
    
    
    
    ####################################################################################################
    ## Compute saliency scores
    mask_img_path_fg, mask_img_path_bg, list_gauss=mask_renderer(torch.load(prep_recorder),140)
    mask_img_path_fg=np.array(mask_img_path_fg)
    mask_img_path_bg=np.array(mask_img_path_bg)
    
    
    
    saliency_score_diffmean=compute_diff_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
    saliency_score_dprime=compute_dprime_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
    saliency_score_aprime=compute_aprime_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
    ####################################################################################################
    
    
    
    
    
    ####################################################################################################
    ## Making the run more memory efficient
    loaded_spliced_model=loaded_spliced_model.to('cpu')
    del GBP
    torch.cuda.empty_cache()
    gc.collect()
    ####################################################################################################
    
    
    
    return saliency_score_diffmean,saliency_score_dprime,saliency_score_aprime

In [52]:
# def compute_saliency(prep_img,prep_recorder,file):
#     prep_img=Variable(prep_img,requires_grad=True).to(device)


#     # Model
#     checkpoint=torch.load(file)
    
#     # loaded_spliced_model=SpliceModel(checkpoint['training_config']['base_model_name'],checkpoint['training_config']['layer_name'],fine_tune=checkpoint['training_config']['fine_tune'],device='cpu')
#     loaded_spliced_model=SpliceModel(checkpoint['training_config']['base_model_name'],checkpoint['training_config']['layer_name'],fine_tune=checkpoint['training_config']['fine_tune'],device=device)

#     loaded_spliced_model.load_state_dict(checkpoint['model_state_dict'],strict=False)
#     GBP = Spliced_GuidedBackprop(loaded_spliced_model,device)



#     # Get gradients
#     guided_grads = GBP.generate_gradients(prep_img, 1)
#     # Gradients
#     guided_grads_vis = guided_grads - guided_grads.min()
#     guided_grads_vis /= guided_grads_vis.max()

#     # Grayscale Gradients
#     grayscale_guided_grads = convert_to_grayscale(guided_grads)
#     grayscale_guided_grads_vis = grayscale_guided_grads - grayscale_guided_grads.min()
#     grayscale_guided_grads_vis /= grayscale_guided_grads_vis.max()


#     # Saliency Score
#     viz_image=grayscale_guided_grads_vis[0]

#     mask_img_path_fg, mask_img_path_bg, list_gauss=mask_renderer(torch.load(prep_recorder),140)
#     mask_img_path_fg=np.array(mask_img_path_fg)
#     mask_img_path_bg=np.array(mask_img_path_bg)
    
    
    
#     saliency_score_diffmean=compute_diff_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
#     saliency_score_dprime=compute_dprime_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
#     saliency_score_aprime=compute_aprime_means(viz_image[np.where(mask_img_path_fg==255)],viz_image[np.where(mask_img_path_bg==255)])
    
    
#     loaded_spliced_model=loaded_spliced_model.to('cpu')
#     del GBP
#     torch.cuda.empty_cache()
#     gc.collect()
    
#     return saliency_score_diffmean,saliency_score_dprime,saliency_score_aprime

## Running the loop - All models, All images

In [ ]:
model_saliency_score_diff={}
model_saliency_score_dprime={}
model_saliency_score_aprime={}


for file in tqdm(model_file_paths):
    print(file)
    checkpoint=torch.load(file)
    
    model_saliency_score_diff[checkpoint['training_config']['layer_name']]=[]
    model_saliency_score_dprime[checkpoint['training_config']['layer_name']]=[]
    model_saliency_score_aprime[checkpoint['training_config']['layer_name']]=[]
    
    for (a, b, d, alpha, nel, labels, record) in tqdm(val_loader_norm,disable=True):
        
        
        for img_num in tqdm(range(a.shape[0]),disable=True):
            prep_img=torch.unsqueeze(a[img_num],0)
            prep_recorder=record[img_num]
            
            saliency_score_diffmean,saliency_score_dprime,saliency_score_aprime=compute_saliency(prep_img,prep_recorder,file)

            model_saliency_score_diff[checkpoint['training_config']['layer_name']].append(saliency_score_diffmean)
            model_saliency_score_dprime[checkpoint['training_config']['layer_name']].append(saliency_score_dprime)
            model_saliency_score_aprime[checkpoint['training_config']['layer_name']].append(saliency_score_aprime)
            
        
        
    model_saliency_score_diff[checkpoint['training_config']['layer_name']]=np.array(model_saliency_score_diff[checkpoint['training_config']['layer_name']])
    model_saliency_score_dprime[checkpoint['training_config']['layer_name']]=np.array(model_saliency_score_dprime[checkpoint['training_config']['layer_name']])
    model_saliency_score_aprime[checkpoint['training_config']['layer_name']]=np.array(model_saliency_score_aprime[checkpoint['training_config']['layer_name']])
    
    

In [ ]:
all_betas=[]
all_labels=[]

for (a, b, d, alpha, nel, labels, record) in tqdm(val_loader_norm,disable=True):
    all_betas.append(b)
    all_labels.append(labels)

    
all_betas=torch.cat(all_betas).numpy()
all_labels=torch.cat(all_labels).numpy()

In [ ]:
model_saliency_score_diff.keys()
model_saliency_score_dprime.keys()
model_saliency_score_aprime.keys()

## Visualizing the scores for all models from the directory

In [ ]:
contour_present_pos=np.where(all_labels==1)[0]
contour_absent_pos=np.where(all_labels==0)[0]


In [ ]:
# plt.figure(figsize=(10,4))
# plt.subplot(1,2,1)
# sns.distplot(model_saliency_score_aprime['avgpool'][contour_present_pos],label='contour_present',color='green')
# sns.distplot(model_saliency_score_aprime['avgpool'][contour_absent_pos],label='contour_absent',color='gray')
# plt.title('saliency_score_aprime')
# plt.legend()

# plt.subplot(1,2,2)
# sns.distplot(model_saliency_score_aprime['avgpool'][contour_present_pos] - model_saliency_score_aprime['avgpool'][contour_absent_pos],label='difference',color='k')
# plt.title('Alignment Sensitivity')
# plt.show()


# plt.show()


In [ ]:
# plt.figure(figsize=(8,10))
# # Scatter Plot with Connecting Lines
# for x1, x2 in zip(model_saliency_score_aprime['avgpool'][contour_absent_pos], model_saliency_score_aprime['avgpool'][contour_present_pos]):
#     plt.plot([x1, x2], [1, 2], 'ko-', markersize=8,alpha=0.09)
    

# plt.yticks([1, 2], ['Contour absent', 'Contour present'])

# plt.gca().spines['top'].set_visible(False)
# plt.gca().spines['right'].set_visible(False)

# plt.show()

### Visualzing all the models

In [ ]:
def get_foreground_sensitivity_allmodels(sal_dict,list_layers,pos):
    mean_sal=[]
    std_sal=[]
    confint_sal=[]
    confint_lower=[]
    confint_upper=[]
    
    
    
    for i in range(len(list_layers)):
        layer=list_layers[i]
        
        
        if layer in sal_dict.keys():
            
            
            mean_sal.append(np.mean(sal_dict[layer][pos]))
            std_sal.append(np.std(sal_dict[layer][pos],ddof=1))
            
            
            
            confint_sal.append(1.96*(np.std(sal_dict[layer][pos],ddof=1) / np.sqrt(len(sal_dict[layer][pos]))))
            confint_lower.append(np.percentile(sal_dict[layer][pos], 2.5))
            confint_upper.append(np.percentile(sal_dict[layer][pos], 97.5))
            
    return mean_sal,std_sal,confint_sal,confint_lower,confint_upper


In [ ]:
def get_alignment_sensitivity_allmodels(sal_dict,list_layers,present_pos,absent_pos):
    
    mean_sal=[]
    std_sal=[]
    confint_sal=[]
    confint_lower=[]
    confint_upper=[]
    
    
    
    for i in range(len(list_layers)):
        layer=list_layers[i]
        
        
        if layer in sal_dict.keys():
            
            
            diff_values=sal_dict[layer][present_pos] - sal_dict[layer][absent_pos]
            
            mean_sal.append(np.mean(diff_values))
            std_sal.append(np.std(diff_values))
            
            
            
            confint_sal.append(1.96*(np.std(diff_values) / np.sqrt(len(diff_values))))
            confint_lower.append(np.percentile(diff_values, 2.5))
            confint_upper.append(np.percentile(diff_values, 97.5))
            
    return mean_sal,std_sal,confint_sal,confint_lower,confint_upper


In [ ]:
list_layers=['features_0',
 'features_1',
 'features_2',
 'features_3',
 'features_4',
 'features_5',
 'features_6',
 'features_7',
 'features_8',
 'features_9',
 'features_10',
 'features_11',
 'features_12',
 'avgpool',
 'classifier_0',
 'classifier_1',
 'classifier_2',
 'classifier_3',
 'classifier_4',
 'classifier_5',
 'classifier_6']

for i in range(len(list_layers)):
    list_layers[i]=list_layers[i].replace('_','.')

In [ ]:
model_saliency_score_aprime['avgpool'].shape

In [ ]:
contour_present_pos=np.where(all_labels==1)[0]
contour_absent_pos=np.where(all_labels==0)[0]

In [ ]:
mean_sal,std_sal,confint_sal,confint_lower,confint_upper=get_foreground_sensitivity_allmodels(model_saliency_score_diff,list_layers,contour_present_pos)



plt.figure(figsize=(22,10))
color_or_finetune=(78/255,121/255,180/255)
spacing_x=2
error_bar_width=3
error_bar_cap=4

plt.errorbar(np.arange(len(mean_sal)) * spacing_x, mean_sal, yerr=confint_sal, 
             capsize=error_bar_cap,elinewidth=error_bar_width,fmt='o',marker='o', color=color_or_finetune, markersize=10, 
             label='or finetune', alpha=0.8)





plt.xticks(np.arange(len(mean_sal)) * spacing_x,list_layers,rotation=90,fontsize=15)
plt.yticks(fontsize=15) 


plt.ylabel('Diff of means',fontsize=15)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.title('Foreground Sensitivity in Contour Images',fontsize=15)

plt.savefig('./dev/manuscript_figures/foreground_sensitivity_allmodel_diffmean.png', format='png', dpi=600)

plt.show()
    
    
    
    

In [ ]:
mean_sal,std_sal,confint_sal,confint_lower,confint_upper=get_foreground_sensitivity_allmodels(model_saliency_score_aprime,list_layers,contour_present_pos)



plt.figure(figsize=(22,10))
color_or_finetune=(78/255,121/255,180/255)
spacing_x=2
error_bar_width=3
error_bar_cap=4

plt.errorbar(np.arange(len(mean_sal)) * spacing_x, mean_sal, yerr=confint_sal, 
             capsize=error_bar_cap,elinewidth=error_bar_width,fmt='o',marker='o', color=color_or_finetune, markersize=10, 
             label='or finetune', alpha=0.8)





plt.xticks(np.arange(len(mean_sal)) * spacing_x,list_layers,rotation=90,fontsize=15)
plt.yticks(fontsize=15) 


plt.ylabel('Aprime',fontsize=15)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.title('Foreground Sensitivity in Contour Images',fontsize=15)

plt.savefig('./dev/manuscript_figures/foreground_sensitivity_allmodel_aprime.png', format='png', dpi=600)
plt.show()
    
    
    
    

In [ ]:
mean_sal,std_sal,confint_sal,confint_lower,confint_upper=get_alignment_sensitivity_allmodels(model_saliency_score_aprime,list_layers,contour_present_pos,contour_absent_pos)

plt.figure(figsize=(22,10))
color_or_finetune=(78/255,121/255,180/255)
spacing_x=2
error_bar_width=3
error_bar_cap=4

plt.errorbar(np.arange(len(mean_sal)) * spacing_x, mean_sal, yerr=confint_sal, 
             capsize=error_bar_cap,elinewidth=error_bar_width,fmt='o',marker='o', color=color_or_finetune, markersize=10, 
             label='or finetune', alpha=0.8)





plt.xticks(np.arange(len(mean_sal)) * spacing_x,list_layers,rotation=90,fontsize=15)
plt.yticks(fontsize=15) 


plt.ylabel('Aprime',fontsize=15)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.title('Alignment Sensitivity in Contour Images',fontsize=15)

plt.savefig('./dev/manuscript_figures/alignment_sensitivity_allmodel_aprime.png', format='png', dpi=600)
plt.show()
    
